In [ ]:
import os
import tensorflow as tf
import numpy as np
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from zipfile import ZipFile
import os


zip_path = '/content/drive/MyDrive/VEG_DATA/Vegetable Images.zip'
extract_dir = '/content/drive/MyDrive/VEG_DATA/Data'

if not os.path.exists(extract_dir):
  os.makedirs(extract_dir)
  print(f"Created extraction directory: {extract_dir}")


with ZipFile(zip_path, 'r') as zip_ref:
  print("Extracting files...")
  for name in zip_ref.namelist():
    print(f"  Extracting: {name}")  # Show individual file extraction
    zip_ref.extract(name, extract_dir)
  print("Extraction complete!")

In [ ]:
def create_image_data_generators(data_dir, batch_size, class_mode='categorical'):


  train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
  )

  validation_test_datagen = ImageDataGenerator(
      rescale=1./255,
  )

  try:
    # Training set
    train_generator = train_datagen.flow_from_directory(
      os.path.join(data_dir, 'train'),
      target_size=(224, 224, 3),
      batch_size=batch_size,
      class_mode=class_mode,
      subset='training',
      seed=42
    )
    # Validation set
    val_generator = validation_test_datagen.flow_from_directory(
      os.path.join(data_dir, 'validation'),
      target_size=(224, 224, 3),
      batch_size=batch_size,
      class_mode=class_mode,
      subset='validation',
      seed=42
    )
    # Test set (no shuffling)
    test_generator = validation_test_datagen.flow_from_directory(
      os.path.join(data_dir, 'test'),
      target_size=(224, 224, 3),
      batch_size=batch_size,
      class_mode=class_mode,
      shuffle=False,
      seed=42
    )

  except FileNotFoundError as e:
    print(f"Error: {e}. Please check your data directory and folder structure.")
    return None

  return train_generator, val_generator, test_generator

In [ ]:
def build_model(input_shape, num_classes, dropout_rate=0.2, is_binary=False, weights='imagenet'):

    # Load EfficientNetB0 pre-trained model with specified weights
    base_model = EfficientNetB0(include_top=False, input_shape=input_shape, weights=weights)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(128, activation='relu')(x)

    if is_binary:
        output = Dense(1, activation='sigmoid')(x)
        loss_func = BinaryCrossentropy()
    else:
        output = Dense(num_classes, activation='softmax')(x)
        loss_func = CategoricalCrossentropy()

    model = Model(inputs=base_model.input, outputs=output)

    # Compile model with Adam optimizer and appropriate loss function
    model.compile(optimizer=Adam(learning_rate=1e-3),
                  loss=loss_func,
                  metrics=['accuracy'])

    return model

In [ ]:
def train_model(model, train_generator, val_generator, epochs, model_path, fine_tune=False):

    early_stop = EarlyStopping(monitor='val_loss', patience=5)
    checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, min_lr=1e-7)

    if fine_tune:
        for layer in model.layers[:-10]:
            layer.trainable = False

    callbacks = [early_stop, checkpoint, reduce_lr]

    history = model.fit(
          train_generator,
          epochs=epochs,
          validation_data=val_generator,
          callbacks=callbacks)

    return history

In [ ]:
def evaluate_model(model, test_generator):

    test_loss, test_acc = model.evaluate(test_generator)
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')

In [ ]:
def predict(model, image_path, class_names):

    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)

    if len(class_names) == 2:
        predicted_class = class_names[int(prediction[0] > 0.5)]
    else:
        predicted_class = class_names[np.argmax(prediction)]

    print(f'Predicted class: {predicted_class}')

In [ ]:
data_dir = '/content/drive/MyDrive/VEG_DATA/Data/Vegetable Images'

image_size = (224, 224, 3)
batch_size = 64
seed = 42
epochs = 20
class_mode='binary'
model_path = '/content/drive/MyDrive/Data/Model/veg_binary_model.h5'

In [ ]:
train_generator, val_generator, test_generator = create_image_data_generators(data_dir, batch_size, class_mode)

Found 5000 images belonging to 5 classes.
Found 0 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.


In [ ]:
train_generator.class_indices

{'Cucumber': 0, 'Papaya': 1, 'Potato': 2, 'Pumpkin': 3, 'Tomato': 4}

In [ ]:
val_generator.class_indices

{'Cucumber': 0, 'Papaya': 1, 'Potato': 2, 'Pumpkin': 3, 'Tomato': 4}

In [ ]:
classes = ['Potato', 'Tomato']  # Customize the classes here

In [ ]:
# Binary Classification
binary_model = build_model(input_shape=image_size, num_classes=2, is_binary=True, weights='imagenet')
binary_history = train_model(binary_model, train_generator, val_generator, epochs, model_path)

Epoch 1/20
19/79 [======>.......................] - ETA: 24:15 - loss: -1069.1670 - accuracy: 0.1908

KeyboardInterrupt: 

**Error debugging time exceeded**

In [ ]:
# Test the binary model
evaluate_model(binary_model, test_generator)

In [ ]:
# Predict on a new image (binary)
test_image_path = '/path/to/test_image.jpg'
predict(binary_model, test_image_path, classes)

In [ ]:
# Multi-class Classification
num_classes = len(classes)
multiclass_model = build_model(input_shape=image_size, num_classes=num_classes, weights='imagenet')
multiclass_history = train_model(multiclass_model, train_generator, val_generator, epochs, model_path)


In [ ]:
# Test the multi-class model
evaluate_model(multiclass_model, test_generator)

In [ ]:
# Predict on a new image (multi-class)
test_image_path = '/path/to/test_image.jpg'
predict(multiclass_model, test_image_path, classes)

In [ ]:

# Fine-tune the multi-class model
fine_tune_history = train_model(multiclass_model, train_generator, val_generator, epochs, model_path, fine_tune=True)